# Speed Skydiving Analysis and Scoring 2019

Analyze one or more FlySight files with speed skydiving data.

This document implements scoring techniques compatible with the FAI World Air Sports Federation [Speed Skydiving Competition Rules, 2019 Edition](https://www.fai.org/sites/default/files/documents/2019_ipc_cr_speedskydiving.pdf) (PDF, 428 KB).

## Environment setup

In [ ]:
from ssscore import COURSE_END
from ssscore import DEG_IN_RAD
from ssscore import FLYSIGHT_SAMPLE_TIME
from ssscore import FLYSIGHT_TIME_FORMAT
from ssscore import RESOURCE_PATH
from ssscore import SPEED_INTERVAL
from ssscore import VALID_MSL

import dateutil.parser
import inspect
import math
import os
import os.path
import shutil

import pandas as pd

import ssscore

### Known drop zones AMSL in meters

The `ssscore` module defines these altitudes; the DZ name corresponds to the symbolic constant, e.g. Bay Area Skydiving ::= `BAY_AREA_SKYDIVING`.  The altitudes were culled from public airport information available on the Worldwide Web.

In [ ]:
from ssscore.elevations import DZElevations # in meters

dir(DZElevations)

#### Set the appropriate DZ elevation

In [ ]:
DZ_AMSL = DZElevations.BAY_AREA_SKYDIVING.value
DZ_AMSL

## FlySight data sources

1. Copy the FlySight `Tracks/YY-MM-dd` directory of interest to the `DATA_LAKE` directory;
   the `DATA_LAKE` can also be an external mount, a Box or Dropbox share, anything that 
   can be mapped to a directory -- even a whole drive!
1. Make a list of all CSV files in the FlySight `DATA_LAKE`
1. Move the CSV to the `DATA_SOURCE` bucket directory

### Define the data lake and data source

![Data sources diagram](images/SSScoring-data-sources.png)
<a id="l_data-def"></a>

In [ ]:
DATA_LAKE   = os.path.join('.', RESOURCE_PATH)
DATA_SOURCE = os.path.join('.', 'data-sources', ssscore.RESOURCE_PATH)

ssscore.updateFlySightDataSource(DATA_LAKE, DATA_SOURCE)

## Top speed and pitch analysis on a single data file

User selects a valid FlySight data path and file name.  Call the `calculateSpeedAndPitch(fileName)` function with this file name.  The function produces:

* Maximum mean speed within a 3-second interval within the course
* Flight pitch in degrees at max speed
* Max speed
* Min speed

All speeds are reported in km/h.

### Explanation of the code in calculateSpeedAndPitchFrom(fileName)

1. Discard all source entries outside of the exit altitude to 1,700 m AGL course
1. Resolve the max speed and pitch within the course
1. Calculate the max mean speed for the 3-second interval near the max speed
1. Return the results in a Series object for later inclusion in a data frame with results from multiple jumps

### Specify the FlySight data file to analyze

In [ ]:
DATA_SOURCE        = os.path.join('.', 'data-sources', ssscore.RESOURCE_PATH)
FLYSIGHT_DATA_FILE = 'FlySight-test-file.csv'

In [ ]:
def _discardDataOutsideCourse(flightData):
    maxHeight       = flightData['hAGL', '(m)'].max()
    height          = flightData['hAGL', '(m)']
    descentVelocity = flightData['velD', '(m/s)']
    flightData      = flightData[(height <= maxHeight) & (height >= COURSE_END) & (descentVelocity >= 0.0)]
    
    return flightData

In [ ]:
import datetime
import time


def _convertToUnixTime(dateString):
    """
    Converts the dateString in FLYSIGHT_TIME_FORMAT into Unix time,
    expressed in hundreths of a second (i.e. 100*timestamp)
    """
    timestamp = datetime.datetime.strptime(dateString, FLYSIGHT_TIME_FORMAT)
    epoch     = datetime.datetime(1970, 1, 1)
    
    return int(100.0*(timestamp-epoch).total_seconds())

In [ ]:
def _selectValidSpeedAnalysisWindowsIn(flightData):
    startTime = flightData['unixTime'].iloc[0]
    stopTime  = flightData['unixTime'].iloc[-1]
    windows   = None
    unixTime  = flightData['unixTime']
    
    for intervalStart in range(startTime, stopTime, FLYSIGHT_SAMPLE_TIME):
        intervalEnd = intervalStart+SPEED_INTERVAL
        window      = flightData[(unixTime >= intervalStart) & (unixTime < intervalEnd)]
        if len(window) == (SPEED_INTERVAL/FLYSIGHT_SAMPLE_TIME):
            if windows is not None:
                windows.append(window)
            else:
                windows = window

    return windows

In [ ]:
def _calculateCourseSpeedUsing(flightData):
    """
    Returns absolute best max speed and 3-second window max speed.
    """
    windows = _selectValidSpeedAnalysisWindowsIn(flightData)
    
    return windows

In [ ]:
def maxHorizontalSpeedFrom(flightData, maxVerticalSpeed):
    velN = flightData[flightData['velD', '(m/s)'] == maxVerticalSpeed]['velN', '(m/s)']
    velE = flightData[flightData['velD', '(m/s)'] == maxVerticalSpeed]['velE', '(m/s)']
    
    return math.sqrt(velN**2+velE**2)   # R vector

In [ ]:
"""
    Adjusts the flight data to compensate for DZ elevation AMSL.
    
    flightData - the raw FlySight data frame
    elevation  - the elevation, in meters, to adjust
    
    All hMSL values in flightData will be offset by +elevation meters.
"""
def adjustElevation(flightData, elevation):
    flightData['hAGL', '(m)'] = flightData['hMSL', '(m)']-elevation
    
    return flightData

In [ ]:
def calculateSpeedAndPitchFor(fileName, elevation = 0.00):
    """
    Accepts a file name to a FlySight data file.
    
    Returns a Series with the results of a speed skydiving jump.
    """
    flightData             = adjustElevation(pd.read_csv(fileName, header = [0, 1]), elevation)
    flightData             = _discardDataOutsideCourse(flightData)
    flightData['unixTime'] = flightData['time'].iloc[:,0].apply(_convertToUnixTime)

    return _calculateCourseSpeedUsing(flightData)

calculateSpeedAndPitchFor(os.path.join(DATA_SOURCE, FLYSIGHT_DATA_FILE), elevation = DZ_AMSL)

## Listing FlySight data files with valid data

This set up generates a list of FlySight data files ready for analysis.  It discards any warm up FlySight data files, those that show no significant changes in elevation MSL across the complete data set.

The list generator uses the `DATA_SOURCE` global variable.  [Change the value of `DATA_SOURCE`](#l_data-def) if necessary.

1. Generate the list of available data files
1. Discard FlySight warm up files by rejecting files without jump data (test:  minimal altitude changes)

![Scoring FlySight files list generation diagram](images/SSScoring-list-scoring-files.png)

In [ ]:
def listDataFilesIn(bucketPath):
    """
    Generate a sorted list of files available in a given bucketPath.
    Files names appear in reverse lexicographical order.
    """

    filesList = pd.Series([os.path.join(bucketPath, fileName) for fileName in sorted(os.listdir(bucketPath), reverse = True) if '.CSV' in fileName or '.csv' in fileName])
    
    return filesList;

In [ ]:
def _hasValidJumpData(fileName):
    flightData = pd.read_csv(fileName, header = [0, 1])['hMSL', '(m)']
    
    return flightData.std() >= VALID_MSL

In [ ]:
def selectValidFlySightFilesFrom(dataFiles):
    included = dataFiles.apply(_hasValidJumpData)
    
    return pd.Series(dataFiles)[included]

In [ ]:
dataFiles = selectValidFlySightFilesFrom(listDataFilesIn(DATA_SOURCE))
dataFiles

## Top speed and pitch analysis on all tracks in the data lake

Takes all the FlySight files in a bucket, detects the ones with valid data, and runs performance analysis over them.  Packs all the results in a data frame, then calculates:

* Average speed
* Max average speed

### Populate data sources

In [ ]:
DATA_LAKE   = './data-lake'
DATA_SOURCE = './data-sources/ciurana'

ssscore.updateFlySightDataSource(DATA_LAKE, DATA_SOURCE)

In [ ]:
# DATA_LAKE   = './data-lake'
# DATA_SOURCE = './data-sources/landgren'
# 
# ssscore.updateFlySightDataSource(DATA_LAKE, DATA_SOURCE)

### Analyze all files in the bucket

In [ ]:
allCompetitionJumps = selectValidFlySightFilesFrom(listDataFilesIn(DATA_SOURCE)).apply(calculateSpeedAndPitchFor)
allCompetitionJumps

### Summary of results

In [ ]:
summary = pd.Series(
            [
                len(allCompetitionJumps),
                allCompetitionJumps['maxSpeed'].mean(),
                allCompetitionJumps['pitch'].mean(),
                allCompetitionJumps['maxSpeed'].max(),
                allCompetitionJumps['pitch'].max(),
            ],
            [
                'totalJumps',
                'meanSpeed',
                'pitch',
                'maxSpeed',
                'maxPitch',
            ])

summary